In [ ]:
# Célula 1: Imports necessários
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision.transforms as transforms
from torchvision import models

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    balanced_accuracy_score, cohen_kappa_score
)
from sklearn.preprocessing import label_binarize

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output

# Verificar GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Dispositivo: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memória: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
print("\n📊 Dataset: EXPW (Expression in-the-Wild)")

In [ ]:
# Célula 2: Configuração Global para EXPW
CONFIG = {
    # Caminhos
    'data_path': '../data/processed/EXPW',  # Caminho para EXPW processado
    'output_path': '../models/expw',
    'results_path': '../results/expw',
    
    # Informações do Dataset
    'dataset_name': 'EXPW',
    'num_classes': 7,
    'emotions': ['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise'],
    'input_size': 224,
    
    # Treinamento
    'batch_size': 32,
    'num_epochs': 50,
    'learning_rate': 0.001,
    'weight_decay': 1e-4,
    
    # Early Stopping
    'patience': 10,
    'min_delta': 0.001,
    
    # Modelos a treinar
    'models_to_train': ['resnet50', 'efficientnet_b0', 'vit_b_16'],
    
    # Configurações avançadas
    'use_weighted_sampler': True,  # Para lidar com desbalanceamento
    'label_smoothing': 0.1,  # Suavização de rótulos
    
    # Seed para reprodutibilidade
    'seed': 42
}

# Criar diretórios
os.makedirs(CONFIG['output_path'], exist_ok=True)
os.makedirs(CONFIG['results_path'], exist_ok=True)

# Setar seeds
torch.manual_seed(CONFIG['seed'])
np.random.seed(CONFIG['seed'])
if torch.cuda.is_available():
    torch.cuda.manual_seed(CONFIG['seed'])
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print("✅ Configurações carregadas para EXPW")
print(f"📁 Dados: {CONFIG['data_path']}")
print(f"💾 Modelos: {CONFIG['output_path']}")
print(f"📊 Resultados: {CONFIG['results_path']}")

In [ ]:
# Célula 3: Classe Dataset para EXPW
from PIL import Image
import cv2

class EXPWDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
        """
        Dataset para carregar imagens pré-processadas do EXPW
        """
        self.root_dir = os.path.join(root_dir, split)
        self.transform = transform
        self.emotions = CONFIG['emotions']
        self.emotion_to_idx = {emotion: idx for idx, emotion in enumerate(self.emotions)}
        
        self.images = []
        self.labels = []
        
        # Carregar caminhos e labels
        for emotion in self.emotions:
            emotion_dir = os.path.join(self.root_dir, emotion)
            if os.path.exists(emotion_dir):
                for img_name in os.listdir(emotion_dir):
                    if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                        self.images.append(os.path.join(emotion_dir, img_name))
                        self.labels.append(self.emotion_to_idx[emotion])
        
        print(f"📂 EXPW {split.upper()} Dataset:")
        print(f"   Total de imagens: {len(self.images)}")
        
        # Contar e mostrar distribuição
        self.class_counts = np.bincount(self.labels, minlength=len(self.emotions))
        self.class_weights = len(self.labels) / (len(self.emotions) * self.class_counts + 1e-6)
        
        print("\n   Distribuição por classe:")
        for idx, emotion in enumerate(self.emotions):
            count = self.class_counts[idx]
            percentage = (count / len(self.labels)) * 100 if len(self.labels) > 0 else 0
            print(f"   {emotion:12s}: {count:5d} imagens ({percentage:5.1f}%)")
            
        # Calcular pesos para balanceamento
        self.sample_weights = [self.class_weights[label] for label in self.labels]
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # Carregar imagem
        img_path = self.images[idx]
        
        try:
            # Ler como grayscale e converter para RGB
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                # Fallback: criar imagem preta se falhar
                image = np.zeros((224, 224), dtype=np.uint8)
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            image = Image.fromarray(image)
        except Exception as e:
            print(f"Erro ao carregar {img_path}: {e}")
            # Criar imagem placeholder
            image = Image.new('RGB', (224, 224), color='black')
        
        if self.transform:
            image = self.transform(image)
        
        label = self.labels[idx]
        return image, label
    
    def get_sample_weights(self):
        return self.sample_weights

print("✅ Classe Dataset EXPW criada")

In [ ]:
# Célula 4: Definir transformações (incluindo augmentation suave)
# Normalização ImageNet
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Transformações para treino (com augmentation suave para EXPW)
train_transform = transforms.Compose([
    transforms.Resize((CONFIG['input_size'], CONFIG['input_size'])),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# Transformações para validação/teste (sem augmentation)
val_transform = transforms.Compose([
    transforms.Resize((CONFIG['input_size'], CONFIG['input_size'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

print("✅ Transformações definidas para EXPW")
print(f"   Input size: {CONFIG['input_size']}x{CONFIG['input_size']}")
print(f"   Normalização: ImageNet")
print(f"   Augmentation no treino: Flip, Rotação (±10°), ColorJitter")

In [ ]:
# Célula 5: Criar datasets e dataloaders com balanceamento
print("🔄 Carregando datasets EXPW...")
print("="*60)

# Criar datasets
train_dataset = EXPWDataset(
    root_dir=CONFIG['data_path'],
    split='train',
    transform=train_transform
)

test_dataset = EXPWDataset(
    root_dir=CONFIG['data_path'],
    split='test',
    transform=val_transform
)

# Configurar sampler balanceado para treino se necessário
train_loader_params = {
    'batch_size': CONFIG['batch_size'],
    'num_workers': 4,
    'pin_memory': True
}

if CONFIG['use_weighted_sampler'] and len(train_dataset) > 0:
    sampler = WeightedRandomSampler(
        weights=train_dataset.get_sample_weights(),
        num_samples=len(train_dataset),
        replacement=True
    )
    train_loader_params['sampler'] = sampler
    print("✅ Usando WeightedRandomSampler para balancear classes")
else:
    train_loader_params['shuffle'] = True

# Criar dataloaders
train_loader = DataLoader(train_dataset, **train_loader_params)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

print(f"\n✅ DataLoaders EXPW criados")
print(f"   Batch size: {CONFIG['batch_size']}")
print(f"   Train batches: {len(train_loader)}")
print(f"   Test batches: {len(test_loader)}")

# Análise de desbalanceamento
if len(train_dataset) > 0:
    imbalance_ratio = max(train_dataset.class_counts) / min(train_dataset.class_counts + [1])
    print(f"   Razão de desbalanceamento: {imbalance_ratio:.2f}x")

In [ ]:
# Célula 6: Visualizar algumas amostras do EXPW
def show_expw_batch(dataloader, num_samples=8):
    """Visualiza um batch de imagens do EXPW"""
    if len(dataloader) == 0:
        print("⚠️ DataLoader vazio")
        return
        
    data_iter = iter(dataloader)
    images, labels = next(data_iter)
    
    # Desnormalizar imagens
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    images_denorm = images * std + mean
    images_denorm = torch.clamp(images_denorm, 0, 1)
    
    fig, axes = plt.subplots(2, 4, figsize=(12, 6))
    axes = axes.flatten()
    
    for i in range(min(num_samples, len(images))):
        img = images_denorm[i].permute(1, 2, 0).numpy()
        axes[i].imshow(img)
        axes[i].set_title(f"{CONFIG['emotions'][labels[i]]}", fontsize=10)
        axes[i].axis('off')
    
    # Esconder eixos não usados
    for i in range(min(num_samples, len(images)), len(axes)):
        axes[i].axis('off')
    
    plt.suptitle('Amostras do Dataset EXPW', fontsize=16)
    plt.tight_layout()
    plt.show()

# Mostrar amostras
if len(train_loader) > 0:
    show_expw_batch(train_loader)
else:
    print("⚠️ Sem dados para visualizar")

In [ ]:
# Célula 7: Definir funções para criar modelos
def create_resnet50(num_classes=7):
    """Cria ResNet50 pré-treinada e ajusta para EXPW"""
    model = models.resnet50(pretrained=True)
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )
    return model

def create_efficientnet_b0(num_classes=7):
    """Cria EfficientNet-B0 pré-treinada para EXPW"""
    model = models.efficientnet_b0(pretrained=True)
    num_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_features, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, num_classes)
    )
    return model

def create_vit_b_16(num_classes=7):
    """Cria Vision Transformer (ViT-B/16) pré-treinado para EXPW"""
    model = models.vit_b_16(pretrained=True)
    num_features = model.heads.head.in_features
    model.heads.head = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )
    return model

# Dicionário de modelos
MODEL_CREATORS = {
    'resnet50': create_resnet50,
    'efficientnet_b0': create_efficientnet_b0,
    'vit_b_16': create_vit_b_16
}

print("✅ Arquiteturas de modelo definidas para EXPW:")
for model_name in CONFIG['models_to_train']:
    print(f"   • {model_name}")

In [ ]:
# Célula 8: Implementar Early Stopping melhorado
class EarlyStopping:
    """Early stopping para evitar overfitting"""
    def __init__(self, patience=7, min_delta=0, verbose=True, mode='min'):
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.mode = mode
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_model_weights = None
        
    def __call__(self, score, model):
        if self.mode == 'min':
            score = -score
            
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f'   EarlyStopping counter: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0
    
    def save_checkpoint(self, model):
        """Salva o estado do modelo"""
        if self.verbose:
            score_str = f"{abs(self.best_score):.4f}" if self.mode == 'min' else f"{self.best_score:.4f}"
            print(f'   Métrica melhorou ({score_str}). Salvando modelo...')
        self.best_model_weights = model.state_dict().copy()
    
    def load_best_weights(self, model):
        """Carrega os melhores pesos"""
        if self.best_model_weights is not None:
            model.load_state_dict(self.best_model_weights)
        return model

print("✅ Early Stopping implementado")
print(f"   Patience: {CONFIG['patience']}")
print(f"   Min delta: {CONFIG['min_delta']}")

In [ ]:
# Célula 9: Funções de treinamento e validação
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
        
    def forward(self, pred, target):
        n_classes = pred.size(-1)
        log_preds = torch.log_softmax(pred, dim=-1)
        loss = -log_preds.sum(dim=-1)
        nll = torch.nn.functional.nll_loss(log_preds, target, reduction='none')
        smooth_loss = loss / n_classes
        loss = (1 - self.smoothing) * nll + self.smoothing * smooth_loss
        return loss.mean()

def train_epoch(model, dataloader, criterion, optimizer, device):
    """Treina o modelo por uma época"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc='Training', leave=False)
    for batch_idx, (images, labels) in enumerate(pbar):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # Estatísticas
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Atualizar barra de progresso
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100*correct/total:.2f}%'
        })
    
    epoch_loss = running_loss / len(dataloader) if len(dataloader) > 0 else 0
    epoch_acc = 100 * correct / total if total > 0 else 0
    
    return epoch_loss, epoch_acc

def validate_epoch(model, dataloader, criterion, device):
    """Valida o modelo"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    all_predictions = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc='Validation', leave=False)
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Guardar para métricas
            probs = torch.softmax(outputs, dim=1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100*correct/total:.2f}%'
            })
    
    epoch_loss = running_loss / len(dataloader) if len(dataloader) > 0 else 0
    epoch_acc = 100 * correct / total if total > 0 else 0
    
    return epoch_loss, epoch_acc, all_predictions, all_labels, all_probs

print("✅ Funções de treinamento com label smoothing definidas")

In [ ]:
# Célula 10: Função principal de treinamento
def train_model(model_name, train_loader, test_loader, device, config):
    """
    Treina um modelo com early stopping e retorna métricas
    """
    print(f"\n{'='*60}")
    print(f"🚀 Treinando {model_name} no EXPW")
    print(f"{'='*60}")
    
    # Verificar se há dados
    if len(train_loader) == 0 or len(test_loader) == 0:
        print("⚠️ Sem dados suficientes para treinar")
        return None, {}, [], [], []
    
    # Criar modelo
    model = MODEL_CREATORS[model_name](num_classes=config['num_classes'])
    model = model.to(device)
    
    # Configurar loss com label smoothing
    if config.get('label_smoothing', 0) > 0:
        criterion = LabelSmoothingCrossEntropy(smoothing=config['label_smoothing'])
        print(f"   Usando Label Smoothing: {config['label_smoothing']}")
    else:
        criterion = nn.CrossEntropyLoss()
    
    # Otimizador com agendador
    optimizer = optim.AdamW(model.parameters(), 
                            lr=config['learning_rate'],
                            weight_decay=config['weight_decay'])
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    early_stopping = EarlyStopping(patience=config['patience'], 
                                  min_delta=config['min_delta'])
    
    # Histórico
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    # Loop de treinamento
    best_val_acc = 0
    for epoch in range(config['num_epochs']):
        print(f"\nEpoch {epoch+1}/{config['num_epochs']}")
        print("-" * 40)
        
        # Treinar
        train_loss, train_acc = train_epoch(model, train_loader, 
                                           criterion, optimizer, device)
        
        # Validar
        val_loss, val_acc, _, _, _ = validate_epoch(model, test_loader, 
                                                    criterion, device)
        
        # Atualizar scheduler
        scheduler.step(val_loss)
        
        # Guardar histórico
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Atualizar melhor accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
        
        # Print estatísticas
        print(f"   Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"   Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"   Best Val Acc: {best_val_acc:.2f}%")
        print(f"   LR: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Early stopping
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("\n⏹️ Early stopping triggered!")
            break
    
    # Carregar melhores pesos
    model = early_stopping.load_best_weights(model)
    
    # Avaliar modelo final
    print("\n📊 Avaliação final no conjunto de teste...")
    val_loss, val_acc, predictions, labels, probs = validate_epoch(
        model, test_loader, criterion, device
    )
    
    print(f"   Final Test Accuracy: {val_acc:.2f}%")
    
    return model, history, predictions, labels, probs

print("✅ Função de treinamento principal configurada para EXPW")

In [ ]:
# Célula 11: Calcular todas as métricas
def calculate_metrics(y_true, y_pred, y_probs, class_names):
    """
    Calcula todas as métricas de avaliação
    """
    metrics = {}
    
    # Métricas básicas
    metrics['accuracy'] = accuracy_score(y_true, y_pred)
    metrics['balanced_accuracy'] = balanced_accuracy_score(y_true, y_pred)
    metrics['cohen_kappa'] = cohen_kappa_score(y_true, y_pred)
    
    # Precision, Recall, F1
    metrics['precision_macro'] = precision_score(y_true, y_pred, average='macro', zero_division=0)
    metrics['precision_weighted'] = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    metrics['recall_macro'] = recall_score(y_true, y_pred, average='macro', zero_division=0)
    metrics['recall_weighted'] = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    metrics['f1_macro'] = f1_score(y_true, y_pred, average='macro', zero_division=0)
    metrics['f1_weighted'] = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    # Métricas por classe
    metrics['precision_per_class'] = precision_score(y_true, y_pred, average=None, zero_division=0)
    metrics['recall_per_class'] = recall_score(y_true, y_pred, average=None, zero_division=0)
    metrics['f1_per_class'] = f1_score(y_true, y_pred, average=None, zero_division=0)
    
    # Matriz de confusão
    metrics['confusion_matrix'] = confusion_matrix(y_true, y_pred)
    
    # ROC-AUC (One-vs-Rest)
    try:
        y_true_bin = label_binarize(y_true, classes=list(range(len(class_names))))
        metrics['roc_auc_ovr'] = roc_auc_score(y_true_bin, y_probs, multi_class='ovr')
        metrics['roc_auc_ovo'] = roc_auc_score(y_true_bin, y_probs, multi_class='ovo')
    except:
        metrics['roc_auc_ovr'] = None
        metrics['roc_auc_ovo'] = None
    
    # Classification report
    metrics['classification_report'] = classification_report(
        y_true, y_pred, target_names=class_names, output_dict=True, zero_division=0
    )
    
    return metrics

def print_metrics_summary(metrics, class_names):
    """
    Imprime resumo das métricas
    """
    print("\n📊 MÉTRICAS DE AVALIAÇÃO - EXPW")
    print("="*60)
    
    print("\n🎯 Métricas Gerais:")
    print(f"   Accuracy:              {metrics['accuracy']:.4f}")
    print(f"   Balanced Accuracy:     {metrics['balanced_accuracy']:.4f}")
    print(f"   Cohen's Kappa:         {metrics['cohen_kappa']:.4f}")
    print(f"   Precision (macro):     {metrics['precision_macro']:.4f}")
    print(f"   Recall (macro):        {metrics['recall_macro']:.4f}")
    print(f"   F1-Score (macro):      {metrics['f1_macro']:.4f}")
    print(f"   Precision (weighted):  {metrics['precision_weighted']:.4f}")
    print(f"   Recall (weighted):     {metrics['recall_weighted']:.4f}")
    print(f"   F1-Score (weighted):   {metrics['f1_weighted']:.4f}")
    
    if metrics['roc_auc_ovr']:
        print(f"   ROC-AUC (OvR):        {metrics['roc_auc_ovr']:.4f}")
        print(f"   ROC-AUC (OvO):        {metrics['roc_auc_ovo']:.4f}")
    
    print("\n📈 Métricas por Classe:")
    print(f"{'Emoção':12s} | {'Precision':>10s} | {'Recall':>10s} | {'F1-Score':>10s}")
    print("-"*50)
    for i, emotion in enumerate(class_names):
        if i < len(metrics['precision_per_class']):
            print(f"{emotion:12s} | {metrics['precision_per_class'][i]:10.4f} | "
                  f"{metrics['recall_per_class'][i]:10.4f} | "
                  f"{metrics['f1_per_class'][i]:10.4f}")

print("✅ Funções de métricas configuradas")

In [ ]:
# Célula 12: Funções de visualização
def plot_training_history(history, model_name):
    """Plota histórico de treinamento"""
    if not history['train_loss']:
        print("⚠️ Sem histórico para plotar")
        return None
        
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Loss
    ax1.plot(history['train_loss'], label='Train Loss', linewidth=2, color='#3498db')
    ax1.plot(history['val_loss'], label='Val Loss', linewidth=2, color='#e74c3c')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title(f'{model_name} - Loss (EXPW)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Accuracy
    ax2.plot(history['train_acc'], label='Train Acc', linewidth=2, color='#2ecc71')
    ax2.plot(history['val_acc'], label='Val Acc', linewidth=2, color='#f39c12')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.set_title(f'{model_name} - Accuracy (EXPW)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle(f'Training History - {model_name} on EXPW', fontsize=16)
    plt.tight_layout()
    return fig

def plot_confusion_matrix(cm, class_names, model_name):
    """Plota matriz de confusão"""
    if cm is None or cm.size == 0:
        print("⚠️ Matriz de confusão vazia")
        return None
        
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Normalizar matriz
    cm_normalized = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 1e-6)
    
    # Plotar
    sns.heatmap(cm_normalized, annot=cm, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Normalized Value'},
                ax=ax)
    
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')
    ax.set_title(f'Confusion Matrix - {model_name} (EXPW)')
    
    # Rotacionar labels se necessário
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    plt.setp(ax.yaxis.get_majorticklabels(), rotation=0)
    
    plt.tight_layout()
    return fig

def plot_metrics_comparison(all_metrics, model_names):
    """Compara métricas entre modelos"""
    metrics_to_plot = ['accuracy', 'balanced_accuracy', 'f1_macro', 'cohen_kappa']
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    colors = ['#3498db', '#2ecc71', '#e74c3c']
    
    for idx, metric in enumerate(metrics_to_plot):
        values = []
        valid_models = []
        for model in model_names:
            if model in all_metrics and metric in all_metrics[model]:
                values.append(all_metrics[model][metric])
                valid_models.append(model)
        
        if values:
            bars = axes[idx].bar(valid_models, values, color=colors[:len(valid_models)])
            axes[idx].set_title(metric.replace('_', ' ').title())
            axes[idx].set_ylim([0, 1])
            axes[idx].set_ylabel('Score')
            
            # Adicionar valores nas barras
            for bar, val in zip(bars, values):
                height = bar.get_height()
                axes[idx].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                              f'{val:.3f}', ha='center', va='bottom')
    
    plt.suptitle('Model Comparison - EXPW Dataset', fontsize=16)
    plt.tight_layout()
    return fig

print("✅ Funções de visualização configuradas")

In [ ]:
# Célula 13: Treinar ResNet50 no EXPW
print("🔄 Iniciando treinamento ResNet50 no EXPW...")

if len(train_loader) > 0 and len(test_loader) > 0:
    resnet_model, resnet_history, resnet_pred, resnet_true, resnet_probs = train_model(
        'resnet50', 
        train_loader, 
        test_loader, 
        device, 
        CONFIG
    )
    
    if resnet_model is not None and len(resnet_pred) > 0:
        # Calcular métricas
        resnet_metrics = calculate_metrics(
            resnet_true, 
            resnet_pred, 
            resnet_probs,
            CONFIG['emotions']
        )
        
        # Mostrar métricas
        print_metrics_summary(resnet_metrics, CONFIG['emotions'])
        
        # Visualizações
        fig1 = plot_training_history(resnet_history, 'ResNet50')
        if fig1:
            plt.show()
        
        fig2 = plot_confusion_matrix(resnet_metrics['confusion_matrix'], 
                                     CONFIG['emotions'], 'ResNet50')
        if fig2:
            plt.show()
        
        # Salvar modelo
        torch.save({
            'model_state_dict': resnet_model.state_dict(),
            'metrics': resnet_metrics,
            'history': resnet_history,
            'config': CONFIG,
            'dataset': 'EXPW'
        }, os.path.join(CONFIG['output_path'], 'resnet50_expw_baseline.pth'))
        
        print("✅ ResNet50 treinado e salvo para EXPW!")
    else:
        print("⚠️ Treinamento ResNet50 falhou ou sem dados")
        resnet_metrics = {}
else:
    print("⚠️ Sem dados suficientes para treinar ResNet50")
    resnet_metrics = {}

In [ ]:
# Célula 14: Treinar EfficientNet-B0 no EXPW
print("🔄 Iniciando treinamento EfficientNet-B0 no EXPW...")

if len(train_loader) > 0 and len(test_loader) > 0:
    efficientnet_model, efficientnet_history, efficientnet_pred, efficientnet_true, efficientnet_probs = train_model(
        'efficientnet_b0', 
        train_loader, 
        test_loader, 
        device, 
        CONFIG
    )
    
    if efficientnet_model is not None and len(efficientnet_pred) > 0:
        # Calcular métricas
        efficientnet_metrics = calculate_metrics(
            efficientnet_true, 
            efficientnet_pred, 
            efficientnet_probs,
            CONFIG['emotions']
        )
        
        # Mostrar métricas
        print_metrics_summary(efficientnet_metrics, CONFIG['emotions'])
        
        # Visualizações
        fig1 = plot_training_history(efficientnet_history, 'EfficientNet-B0')
        if fig1:
            plt.show()
        
        fig2 = plot_confusion_matrix(efficientnet_metrics['confusion_matrix'], 
                                     CONFIG['emotions'], 'EfficientNet-B0')
        if fig2:
            plt.show()
        
        # Salvar modelo
        torch.save({
            'model_state_dict': efficientnet_model.state_dict(),
            'metrics': efficientnet_metrics,
            'history': efficientnet_history,
            'config': CONFIG,
            'dataset': 'EXPW'
        }, os.path.join(CONFIG['output_path'], 'efficientnet_b0_expw_baseline.pth'))
        
        print("✅ EfficientNet-B0 treinado e salvo para EXPW!")
    else:
        print("⚠️ Treinamento EfficientNet-B0 falhou ou sem dados")
        efficientnet_metrics = {}
else:
    print("⚠️ Sem dados suficientes para treinar EfficientNet-B0")
    efficientnet_metrics = {}

In [ ]:
# Célula 15: Treinar Vision Transformer no EXPW
print("🔄 Iniciando treinamento Vision Transformer no EXPW...")

if len(train_loader) > 0 and len(test_loader) > 0:
    vit_model, vit_history, vit_pred, vit_true, vit_probs = train_model(
        'vit_b_16', 
        train_loader, 
        test_loader, 
        device, 
        CONFIG
    )
    
    if vit_model is not None and len(vit_pred) > 0:
        # Calcular métricas
        vit_metrics = calculate_metrics(
            vit_true, 
            vit_pred, 
            vit_probs,
            CONFIG['emotions']
        )
        
        # Mostrar métricas
        print_metrics_summary(vit_metrics, CONFIG['emotions'])
        
        # Visualizações
        fig1 = plot_training_history(vit_history, 'ViT-B/16')
        if fig1:
            plt.show()
        
        fig2 = plot_confusion_matrix(vit_metrics['confusion_matrix'], 
                                     CONFIG['emotions'], 'ViT-B/16')
        if fig2:
            plt.show()
        
        # Salvar modelo
        torch.save({
            'model_state_dict': vit_model.state_dict(),
            'metrics': vit_metrics,
            'history': vit_history,
            'config': CONFIG,
            'dataset': 'EXPW'
        }, os.path.join(CONFIG['output_path'], 'vit_b_16_expw_baseline.pth'))
        
        print("✅ Vision Transformer treinado e salvo para EXPW!")
    else:
        print("⚠️ Treinamento ViT falhou ou sem dados")
        vit_metrics = {}
else:
    print("⚠️ Sem dados suficientes para treinar ViT")
    vit_metrics = {}

In [ ]:
# Célula 16: Comparar todos os modelos no EXPW
print("\n" + "="*60)
print("🏆 COMPARAÇÃO DE MODELOS - EXPW (BASELINE)")
print("="*60)

# Consolidar métricas
all_metrics = {}
if 'resnet_metrics' in locals() and resnet_metrics:
    all_metrics['ResNet50'] = resnet_metrics
if 'efficientnet_metrics' in locals() and efficientnet_metrics:
    all_metrics['EfficientNet-B0'] = efficientnet_metrics
if 'vit_metrics' in locals() and vit_metrics:
    all_metrics['ViT-B/16'] = vit_metrics

if all_metrics:
    # Criar tabela comparativa
    comparison_data = []
    for model_name, metrics in all_metrics.items():
        comparison_data.append({
            'Model': model_name,
            'Dataset': 'EXPW',
            'Accuracy': f"{metrics.get('accuracy', 0):.4f}",
            'Balanced Acc': f"{metrics.get('balanced_accuracy', 0):.4f}",
            'Precision (macro)': f"{metrics.get('precision_macro', 0):.4f}",
            'Recall (macro)': f"{metrics.get('recall_macro', 0):.4f}",
            'F1-Score (macro)': f"{metrics.get('f1_macro', 0):.4f}",
            'F1-Score (weighted)': f"{metrics.get('f1_weighted', 0):.4f}",
            'Cohen Kappa': f"{metrics.get('cohen_kappa', 0):.4f}"
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    print("\n📊 Tabela Comparativa - EXPW:")
    print(df_comparison.to_string(index=False))
    
    # Identificar melhor modelo
    best_model = max(all_metrics.items(), 
                     key=lambda x: x[1].get('balanced_accuracy', 0))
    print(f"\n🥇 Melhor modelo (por balanced accuracy): {best_model[0]} - {best_model[1].get('balanced_accuracy', 0):.4f}")
    
    # Plotar comparação
    if len(all_metrics) > 0:
        fig = plot_metrics_comparison(all_metrics, list(all_metrics.keys()))
        if fig:
            plt.show()
else:
    print("⚠️ Nenhuma métrica disponível para comparação")
    df_comparison = pd.DataFrame()

In [ ]:
# Célula 17: Análise detalhada por classe no EXPW
def analyze_class_performance_expw(all_metrics, emotions):
    """Analisa performance por classe em todos os modelos para EXPW"""
    
    if not all_metrics:
        print("⚠️ Sem métricas para análise")
        return None
        
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, metric_type in enumerate(['precision_per_class', 'recall_per_class', 'f1_per_class']):
        ax = axes[idx]
        
        x = np.arange(len(emotions))
        width = 0.25
        colors = ['#3498db', '#2ecc71', '#e74c3c']
        
        for i, (model_name, metrics) in enumerate(all_metrics.items()):
            if metric_type in metrics:
                values = metrics[metric_type]
                if len(values) > 0:
                    ax.bar(x + i*width, values, width, 
                          label=model_name, color=colors[i % len(colors)])
        
        ax.set_xlabel('Emotion')
        ax.set_ylabel('Score')
        ax.set_title(metric_type.replace('_', ' ').title().replace('Per Class', ''))
        ax.set_xticks(x + width)
        ax.set_xticklabels(emotions, rotation=45)
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.suptitle('Performance by Emotion Class - EXPW Dataset', fontsize=16)
    plt.tight_layout()
    return fig

# Executar análise
if all_metrics:
    fig = analyze_class_performance_expw(all_metrics, CONFIG['emotions'])
    if fig:
        plt.show()
    
    # Identificar classes problemáticas
    print("\n🔍 Análise de Classes Problemáticas no EXPW:")
    for emotion_idx, emotion in enumerate(CONFIG['emotions']):
        print(f"\n{emotion}:")
        for model_name, metrics in all_metrics.items():
            if 'f1_per_class' in metrics and emotion_idx < len(metrics['f1_per_class']):
                f1 = metrics['f1_per_class'][emotion_idx]
                print(f"   {model_name:15s}: F1={f1:.4f}")

In [ ]:
# Célula 18: Salvar todos os resultados EXPW para comparação futura
import pickle

# Consolidar todos os resultados
expw_baseline_results = {
    'timestamp': datetime.now().isoformat(),
    'dataset': 'EXPW',
    'config': CONFIG,
    'models': {},
    'comparison': df_comparison.to_dict() if not df_comparison.empty else {}
}

# Adicionar resultados de cada modelo se existirem
if 'resnet_metrics' in locals() and resnet_metrics:
    expw_baseline_results['models']['resnet50'] = {
        'metrics': resnet_metrics,
        'history': resnet_history if 'resnet_history' in locals() else {},
    }

if 'efficientnet_metrics' in locals() and efficientnet_metrics:
    expw_baseline_results['models']['efficientnet_b0'] = {
        'metrics': efficientnet_metrics,
        'history': efficientnet_history if 'efficientnet_history' in locals() else {},
    }

if 'vit_metrics' in locals() and vit_metrics:
    expw_baseline_results['models']['vit_b_16'] = {
        'metrics': vit_metrics,
        'history': vit_history if 'vit_history' in locals() else {},
    }

# Salvar em diferentes formatos
# 1. Pickle (preserva tudo)
with open(os.path.join(CONFIG['results_path'], 'expw_baseline_results.pkl'), 'wb') as f:
    pickle.dump(expw_baseline_results, f)

# 2. JSON (apenas métricas principais)
json_results = {
    'timestamp': expw_baseline_results['timestamp'],
    'dataset': 'EXPW',
    'config': {k: v for k, v in CONFIG.items() if not callable(v)},
    'models': {}
}

for model_name, model_data in expw_baseline_results['models'].items():
    if 'metrics' in model_data:
        json_results['models'][model_name] = {
            'accuracy': float(model_data['metrics'].get('accuracy', 0)),
            'balanced_accuracy': float(model_data['metrics'].get('balanced_accuracy', 0)),
            'precision_macro': float(model_data['metrics'].get('precision_macro', 0)),
            'recall_macro': float(model_data['metrics'].get('recall_macro', 0)),
            'f1_macro': float(model_data['metrics'].get('f1_macro', 0)),
            'f1_weighted': float(model_data['metrics'].get('f1_weighted', 0)),
            'cohen_kappa': float(model_data['metrics'].get('cohen_kappa', 0))
        }

with open(os.path.join(CONFIG['results_path'], 'expw_baseline_metrics.json'), 'w') as f:
    json.dump(json_results, f, indent=2)

# 3. CSV com métricas principais
if not df_comparison.empty:
    df_comparison.to_csv(os.path.join(CONFIG['results_path'], 'expw_baseline_comparison.csv'), index=False)

print("✅ Resultados EXPW salvos em:")
print(f"   • {CONFIG['results_path']}/expw_baseline_results.pkl")
print(f"   • {CONFIG['results_path']}/expw_baseline_metrics.json")
if not df_comparison.empty:
    print(f"   • {CONFIG['results_path']}/expw_baseline_comparison.csv")

In [ ]:
# Célula 19: Gerar relatório final EXPW
def generate_expw_report(all_metrics, config):
    """Gera relatório em texto com todos os resultados do EXPW"""
    
    report = []
    report.append("="*80)
    report.append("RELATÓRIO DE TREINAMENTO - MODELOS BASELINE EXPW")
    report.append("="*80)
    report.append(f"\nData: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report.append(f"Dataset: EXPW (Expression in-the-Wild)")
    report.append(f"Caminho: {config['data_path']}")
    report.append(f"Dispositivo: {device}")
    
    report.append("\n" + "-"*80)
    report.append("CONFIGURAÇÕES DE TREINAMENTO:")
    report.append("-"*80)
    report.append(f"Batch Size: {config['batch_size']}")
    report.append(f"Learning Rate: {config['learning_rate']}")
    report.append(f"Weight Decay: {config['weight_decay']}")
    report.append(f"Max Epochs: {config['num_epochs']}")
    report.append(f"Early Stopping Patience: {config['patience']}")
    report.append(f"Input Size: {config['input_size']}x{config['input_size']}")
    report.append(f"Label Smoothing: {config.get('label_smoothing', 0)}")
    report.append(f"Weighted Sampler: {config.get('use_weighted_sampler', False)}")
    
    if all_metrics:
        report.append("\n" + "-"*80)
        report.append("RESULTADOS POR MODELO:")
        report.append("-"*80)
        
        for model_name, metrics in all_metrics.items():
            report.append(f"\n### {model_name}")
            report.append(f"   Accuracy: {metrics.get('accuracy', 0):.4f}")
            report.append(f"   Balanced Accuracy: {metrics.get('balanced_accuracy', 0):.4f}")
            report.append(f"   Cohen's Kappa: {metrics.get('cohen_kappa', 0):.4f}")
            report.append(f"   Precision (macro): {metrics.get('precision_macro', 0):.4f}")
            report.append(f"   Recall (macro): {metrics.get('recall_macro', 0):.4f}")
            report.append(f"   F1-Score (macro): {metrics.get('f1_macro', 0):.4f}")
            report.append(f"   F1-Score (weighted): {metrics.get('f1_weighted', 0):.4f}")
            if metrics.get('roc_auc_ovr'):
                report.append(f"   ROC-AUC (OvR): {metrics['roc_auc_ovr']:.4f}")
        
        # Melhor modelo
        if all_metrics:
            best_model = max(all_metrics.items(), 
                           key=lambda x: x[1].get('balanced_accuracy', 0))
            report.append("\n" + "-"*80)
            report.append("MELHOR MODELO:")
            report.append("-"*80)
            report.append(f"Modelo: {best_model[0]}")
            report.append(f"Balanced Accuracy: {best_model[1].get('balanced_accuracy', 0):.4f}")
        
        # Classes problemáticas
        report.append("\n" + "-"*80)
        report.append("ANÁLISE POR CLASSE (F1-Score):")
        report.append("-"*80)
        
        for emotion_idx, emotion in enumerate(config['emotions']):
            report.append(f"\n{emotion}:")
            for model_name, metrics in all_metrics.items():
                if 'f1_per_class' in metrics and emotion_idx < len(metrics['f1_per_class']):
                    f1 = metrics['f1_per_class'][emotion_idx]
                    report.append(f"   {model_name}: {f1:.4f}")
        
        # Identificar classe mais difícil
        avg_f1_per_class = {}
        for emotion_idx, emotion in enumerate(config['emotions']):
            f1_scores = []
            for m in all_metrics.values():
                if 'f1_per_class' in m and emotion_idx < len(m['f1_per_class']):
                    f1_scores.append(m['f1_per_class'][emotion_idx])
            if f1_scores:
                avg_f1_per_class[emotion] = np.mean(f1_scores)
        
        if avg_f1_per_class:
            hardest_class = min(avg_f1_per_class.items(), key=lambda x: x[1])
            easiest_class = max(avg_f1_per_class.items(), key=lambda x: x[1])
            
            report.append("\n" + "-"*80)
            report.append("INSIGHTS:")
            report.append("-"*80)
            report.append(f"Classe mais fácil: {easiest_class[0]} (F1 médio: {easiest_class[1]:.4f})")
            report.append(f"Classe mais difícil: {hardest_class[0]} (F1 médio: {hardest_class[1]:.4f})")
    else:
        report.append("\n⚠️ Nenhum modelo foi treinado com sucesso")
    
    report.append("\n" + "="*80)
    report.append("FIM DO RELATÓRIO")
    report.append("="*80)
    
    return "\n".join(report)

# Gerar e salvar relatório
report = generate_expw_report(all_metrics if 'all_metrics' in locals() else {}, CONFIG)
print(report)

# Salvar relatório em arquivo
with open(os.path.join(CONFIG['results_path'], 'expw_baseline_report.txt'), 'w') as f:
    f.write(report)

print(f"\n📄 Relatório salvo em: {CONFIG['results_path']}/expw_baseline_report.txt")

In [ ]:
# Célula 20: Preparar estrutura para comparação futura com versão augmented
comparison_structure = {
    'baseline': {
        'timestamp': datetime.now().isoformat(),
        'dataset': 'EXPW',
        'data_type': 'original_preprocessed',
        'models': {}
    },
    'augmented': {
        'timestamp': None,
        'dataset': 'EXPW',
        'data_type': 'with_augmentation',
        'models': {}
    }
}

# Preencher com resultados baseline
for model_key, model_name in [('resnet50', 'ResNet50'), 
                               ('efficientnet_b0', 'EfficientNet-B0'), 
                               ('vit_b_16', 'ViT-B/16')]:
    if model_name in all_metrics:
        model_metrics = all_metrics[model_name]
        comparison_structure['baseline']['models'][model_key] = {
            'accuracy': float(model_metrics.get('accuracy', 0)),
            'balanced_accuracy': float(model_metrics.get('balanced_accuracy', 0)),
            'f1_macro': float(model_metrics.get('f1_macro', 0)),
            'f1_weighted': float(model_metrics.get('f1_weighted', 0)),
            'cohen_kappa': float(model_metrics.get('cohen_kappa', 0))
        }

# Salvar estrutura para futura comparação
with open(os.path.join(CONFIG['results_path'], 'expw_comparison_structure.json'), 'w') as f:
    json.dump(comparison_structure, f, indent=2)

print("✅ Estrutura de comparação EXPW preparada!")
print("📌 Use este arquivo para comparar com resultados augmentados:")
print(f"   {CONFIG['results_path']}/expw_comparison_structure.json")

print("\n" + "="*60)
print("🎉 TREINAMENTO BASELINE EXPW COMPLETO!")
print("="*60)
print("\nPróximos passos:")
print("1. Implementar data augmentation avançada para EXPW")
print("2. Treinar modelos com dados augmentados")
print("3. Comparar resultados usando a estrutura salva")
print("4. Analisar melhorias obtidas com augmentation")
print("5. Considerar técnicas específicas para datasets desbalanceados")

# Resumo final
if all_metrics:
    print("\n📊 Resumo dos Resultados:")
    for model_name, metrics in all_metrics.items():
        print(f"\n{model_name}:")
        print(f"  • Accuracy: {metrics.get('accuracy', 0):.2%}")
        print(f"  • F1-Score (macro): {metrics.get('f1_macro', 0):.2%}")